# 基于DNN的花卉识别练习

导入需要的包

In [1]:
# “skimage” python 图像处理模块，如果用的conda的虚拟环境，需要用conda install scikit-image 进行安装
# glob 文件路径查找模块
# cv2图像处理包，比skimage要快
import cv2
from skimage import io,transform
import glob 
import os
import tensorflow as tf
import numpy as np

数据集路径---根据自己的路径进行改写

In [2]:
path='/home/liuyanfang/lyf/Codes/GAN/flower_photos/'

由于我们的数据集图片大小不一致，所以需要resize成统一大小 这里resize成100x100x3

In [3]:
w=100
h=100
c=3


读取数据集图片并添加标签,最后的形式是data 对应图片, label 是标签，roses 0,daisy 1,sunflowers 2,tulips 3,dandelion 4.

In [4]:
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]#逐个打开文件夹，并提取图片
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):#用于在for循环中得到计数
        for im in glob.glob(folder+'/*.jpg'):#返回所有匹配的文件路径列表
            #print('reading the images:%s'%(im))
            img=io.imread(im)
            img=transform.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
                  
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)#变成矩阵格式
data,label=read_img(path)

/home/yuyue/anaconda2/envs/tensorflow-gpu/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [4]:
def read_img(path):
    cate=[path+x for x in os.listdir(path) if os.path.isdir(path+x)]#逐个打开文件夹，并提取图片
    imgs=[]
    labels=[]
    for idx,folder in enumerate(cate):#用于在for循环中得到计数
        for im in os.listdir(folder):#返回所有匹配的文件路径列表        
       
            img=cv2.imread(folder+'/'+im)
            #print('reading the images:%s'%(folder+'/'+im))
            img=cv2.resize(img,(w,h))
            imgs.append(img)
            labels.append(idx)
                  
    return np.asarray(imgs,np.float32),np.asarray(labels,np.int32)#变成矩阵格式
data,label=read_img(path)

将数据集打乱顺序，下阶段课程会讲其他方法

In [5]:
num_example=data.shape[0] # data.shape是(3029, 100, 100, 3)
arr=np.arange(num_example)# 创建等差数组 0，1，...,3028
np.random.shuffle(arr)# 打乱顺序
data=data[arr]
label=label[arr]
print(label)


[3 2 3 ... 1 1 2]


将标签onehot

In [6]:
def to_one_hot(labels):
    l = len(labels)
    res = np.zeros((l, 5), dtype=np.float32)
    for i in range(l):
        res[i][labels[i]] = 1.
    return res
label_oh = to_one_hot(label)


将所有数据集分为训练集80%、验证集10%和测试集10%

In [7]:
ratio=0.8
s=np.int(num_example*ratio)
x_train=data[:s]
y_train=label_oh[:s]
x_val=data[s:]
y_val=label_oh[s:]

生成batch

In [8]:
def gen_batch(dataset, labelset, batchsize):
    for i in range(dataset.shape[0]//batchsize):
        pos = i * batchsize
        x = dataset[pos:pos + batchsize]
        y = labelset[pos:pos + batchsize]
        yield x,y
    remain = np.shape(dataset)[0] % batchsize
    if remain != 0:
        x = dataset[-remain:]
        y = labelset[-remain:]
        yield x,y

构建网络

In [9]:
graph = tf.Graph()
with graph.as_default():
    X=tf.placeholder(tf.float32,shape=[None,w,h,c],name='X')
    y_=tf.placeholder(tf.int32,shape=[None,5],name='y_')#keep_prob = tf.placeholder(tf.float32)
    

In [10]:
with graph.as_default():
    with tf.name_scope('hidden_layers'):
        re1 = tf.reshape(X, [-1, w * h* c])
        dense1 = tf.layers.dense(inputs=re1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        dense2 = tf.layers.dense(inputs=dense1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        dense3= tf.layers.dense(inputs=dense2,  units=128,  activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    with tf.name_scope("fully_connected"):
        logits= tf.layers.dense(inputs=dense3,  units=5,   activation=None,
                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))

In [13]:
with graph.as_default():
    with tf.name_scope("cal_loss"):
        #loss = tf.losses.sparse_softmax_cross_entropy(labels=y_,logits=logits)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=logits))
    with tf.name_scope("train"):
        train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    with tf.name_scope("cal_accuracy"):
        true = tf.argmax(y_, axis=1)
        correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int64), true)    
        acc= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    summary_loss = tf.summary.scalar('cost', loss)
    summary_acc = tf.summary.scalar('accuracy', acc)

训练和测试数据

In [12]:
epoch=200
batch_size=64
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp/visualization/',sess.graph)
    saver = tf.train.Saver()
    
    step = 0
    for epc in range(epoch):
        for x, y in gen_batch(x_train, y_train, batch_size):
            l, ac, _,s_m = sess.run([loss, acc, train_op, merged], feed_dict={X:x,y_:y})
            if step % 500 == 0:
                print("Step: {:>4}, Loss: {:.4f}, Acc: {:.4%}".format(step, l, ac))
            step += 1
            writer.add_summary(s_m, global_step=epc)
    print("Training finished.")
    l, ac, _ = sess.run([loss, acc, train_op],
                                       {X: x_val, y_: y_val})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))

Step:    0, Loss: 1.6095, Acc: 20.3125%
Step:  500, Loss: 1.1360, Acc: 53.1250%
Step: 1000, Loss: 0.6236, Acc: 71.8750%
Step: 1500, Loss: 0.3960, Acc: 78.1250%
Step: 2000, Loss: 0.5905, Acc: 76.5625%
Step: 2500, Loss: 0.5793, Acc: 78.1250%
Step: 3000, Loss: 0.3727, Acc: 85.9375%
Step: 3500, Loss: 0.3333, Acc: 85.9375%
Step: 4000, Loss: 0.4079, Acc: 89.0625%
Step: 4500, Loss: 0.0735, Acc: 95.3125%
Step: 5000, Loss: 0.1718, Acc: 93.7500%
Step: 5500, Loss: 0.2913, Acc: 92.1875%
Step: 6000, Loss: 0.0870, Acc: 96.8750%
Step: 6500, Loss: 0.0367, Acc: 98.4375%
Step: 7000, Loss: 0.0660, Acc: 96.8750%
Step: 7500, Loss: 0.0646, Acc: 96.8750%
Step: 8000, Loss: 0.5751, Acc: 89.0625%
Step: 8500, Loss: 0.0086, Acc: 100.0000%
Step: 9000, Loss: 0.2588, Acc: 95.3125%
Training finished.
Testing Loss: 5.1341, Testing Acc: 47.9564%


用防止过拟合方法 对结果影响不大

# 优化模型

### 添加正则化

In [10]:
with graph.as_default():
    with tf.name_scope('hidden_layers'):
        re1 = tf.reshape(X, [-1, w * h* c])
        dense1 = tf.layers.dense(inputs=re1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
        dense2 = tf.layers.dense(inputs=dense1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
        dense3= tf.layers.dense(inputs=dense2,  units=128,  activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
    with tf.name_scope("fully_connected"):
        logits= tf.layers.dense(inputs=dense3,  units=5,   activation=None,
                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))

In [12]:
epoch=200
batch_size=64
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp/visualization_l2/',sess.graph)
    saver = tf.train.Saver()
    
    step = 0
    for epc in range(epoch):
        for x, y in gen_batch(x_train, y_train, batch_size):
            l, ac, _,s_m = sess.run([loss, acc, train_op, merged], feed_dict={X:x,y_:y})
            if step % 500 == 0:
                print("Step: {:>4}, Loss: {:.4f}, Acc: {:.4%}".format(step, l, ac))
            step += 1
            writer.add_summary(s_m, global_step=epc)
    print("Training finished.")
    l, ac, _ = sess.run([loss, acc, train_op],
                                       {X: x_val, y_: y_val})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))

Step:    0, Loss: 1.6094, Acc: 23.4375%
Step:  500, Loss: 1.0235, Acc: 67.1875%
Step: 1000, Loss: 0.7885, Acc: 65.6250%
Step: 1500, Loss: 0.5379, Acc: 79.6875%
Step: 2000, Loss: 0.3865, Acc: 85.9375%
Step: 2500, Loss: 0.3553, Acc: 85.9375%
Step: 3000, Loss: 0.2125, Acc: 90.6250%
Step: 3500, Loss: 0.0487, Acc: 98.4375%
Step: 4000, Loss: 0.1525, Acc: 96.8750%
Step: 4500, Loss: 0.0308, Acc: 100.0000%
Step: 5000, Loss: 0.1623, Acc: 92.1875%
Step: 5500, Loss: 0.0071, Acc: 100.0000%
Step: 6000, Loss: 0.0961, Acc: 96.8750%
Step: 6500, Loss: 0.1535, Acc: 95.3125%
Step: 7000, Loss: 0.0094, Acc: 100.0000%
Step: 7500, Loss: 0.0632, Acc: 98.4375%
Step: 8000, Loss: 0.0135, Acc: 100.0000%
Step: 8500, Loss: 0.5829, Acc: 85.9375%
Step: 9000, Loss: 0.0478, Acc: 98.4375%
Training finished.
Testing Loss: 4.1709, Testing Acc: 46.8665%



### 添加dropout层


In [12]:
with graph.as_default():
    keep_prob = tf.placeholder(tf.float32)
    with tf.name_scope('hidden_layers'):
        re1 = tf.reshape(X, [-1, w * h* c])
        dense1 = tf.layers.dense(inputs=re1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        dense2 = tf.layers.dense(inputs=dense1,      units=1024,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        dense3= tf.layers.dense(inputs=dense2,  units=128,  activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
        drop_out=tf.nn.dropout(dense3,keep_prob)
    with tf.name_scope("fully_connected"):
        logits= tf.layers.dense(inputs=drop_out,  units=5,   activation=None,
                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))

In [17]:
epoch=200
batch_size=64
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp/visualization_l2/',sess.graph)
    saver = tf.train.Saver()
    
    step = 0
    for epc in range(epoch):
        for x, y in gen_batch(x_train, y_train, batch_size):
            l, ac, _,s_m = sess.run([loss, acc, train_op, merged], feed_dict={X:x,y_:y,keep_prob:1})
            if step % 500 == 0:
                print("Step: {:>4}, Loss: {:.4f}, Acc: {:.4%}".format(step, l, ac))
            step += 1
            writer.add_summary(s_m, global_step=epc)
    print("Training finished.")
    l, ac, _ = sess.run([loss, acc, train_op],
                                       {X: x_val, y_: y_val,keep_prob:1})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))

Step:    0, Loss: 1.6076, Acc: 12.5000%
Step:  500, Loss: 1.5767, Acc: 35.9375%
Step: 1000, Loss: 1.6033, Acc: 23.4375%
Step: 1500, Loss: 1.6399, Acc: 14.0625%
Step: 2000, Loss: 1.6184, Acc: 26.5625%
Step: 2500, Loss: 1.5844, Acc: 28.1250%
Step: 3000, Loss: 1.5798, Acc: 26.5625%
Step: 3500, Loss: 1.5860, Acc: 26.5625%
Step: 4000, Loss: 1.6117, Acc: 18.7500%
Step: 4500, Loss: 1.5901, Acc: 32.8125%
Step: 5000, Loss: 1.6385, Acc: 17.1875%
Step: 5500, Loss: 1.5761, Acc: 31.2500%
Step: 6000, Loss: 1.6078, Acc: 25.0000%
Step: 6500, Loss: 1.6060, Acc: 23.4375%
Step: 7000, Loss: 1.5934, Acc: 26.5625%
Step: 7500, Loss: 1.6114, Acc: 18.7500%
Step: 8000, Loss: 1.6166, Acc: 18.7500%
Step: 8500, Loss: 1.5769, Acc: 29.6875%
Step: 9000, Loss: 1.5873, Acc: 28.1250%
Training finished.
Testing Loss: 1.6103, Testing Acc: 22.0708%


### 修改学习率


查看tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate,
                      staircase=False, name=None) 
使用方法

In [11]:
with graph.as_default():
    learning_rate=0.001
    global_steps = tf.Variable(0, trainable=False)
    learn_rate = tf.train.exponential_decay(learning_rate,global_steps,1000,0.9,staircase=True)
    with tf.name_scope("cal_loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=logits))
    with tf.name_scope("train"):
        train_op=tf.train.AdamOptimizer(learning_rate).minimize(loss,global_step=global_steps)
    with tf.name_scope("cal_accuracy"):
        true = tf.argmax(y_, axis=1)
        correct_prediction = tf.equal(tf.cast(tf.argmax(logits,1),tf.int64), true)    
        acc= tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    summary_loss = tf.summary.scalar('cost', loss)
    summary_acc = tf.summary.scalar('accuracy', acc)

In [14]:
epoch=200
batch_size=64
with tf.Session(graph=graph) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('./tmp/visualization_l2/',sess.graph)
    saver = tf.train.Saver()
    
    step = 0
    for epc in range(epoch):
        for x, y in gen_batch(x_train, y_train, batch_size):
            l, ac, Learn_rate,_,s_m = sess.run([loss, acc,learn_rate, train_op, merged], feed_dict={X:x,y_:y})
            if step % 500 == 0:
                print("Step: {:>4}, Learn_rate: {:.4f}, Loss: {:.4f},  Acc: {:.4%}".format(step, Learn_rate , l, ac))
            step += 1
            writer.add_summary(s_m, global_step=epc)
    print("Training finished.")
    l, ac, _ = sess.run([loss, acc, train_op],
                                       {X: x_val, y_: y_val})
    print("Testing Loss: {:.4f}, Testing Acc: {:.4%}".format(l, ac))

Step:    0, Learn_rate: 0.0010, Loss: 1.6097,  Acc: 1.5625%
Step:  500, Learn_rate: 0.0010, Loss: 0.6427,  Acc: 71.8750%
Step: 1000, Learn_rate: 0.0009, Loss: 0.3524,  Acc: 85.9375%
Step: 1500, Learn_rate: 0.0009, Loss: 0.2983,  Acc: 85.9375%
Step: 2000, Learn_rate: 0.0008, Loss: 0.1944,  Acc: 92.1875%
Step: 2500, Learn_rate: 0.0008, Loss: 0.4052,  Acc: 85.9375%
Step: 3000, Learn_rate: 0.0007, Loss: 0.1683,  Acc: 92.1875%
Step: 3500, Learn_rate: 0.0007, Loss: 0.0226,  Acc: 100.0000%
Step: 4000, Learn_rate: 0.0007, Loss: 0.2541,  Acc: 96.8750%
Step: 4500, Learn_rate: 0.0007, Loss: 0.0855,  Acc: 96.8750%
Step: 5000, Learn_rate: 0.0006, Loss: 0.0464,  Acc: 98.4375%
Step: 5500, Learn_rate: 0.0006, Loss: 0.0072,  Acc: 100.0000%
Step: 6000, Learn_rate: 0.0005, Loss: 0.0359,  Acc: 98.4375%
Step: 6500, Learn_rate: 0.0005, Loss: 0.0031,  Acc: 100.0000%
Step: 7000, Learn_rate: 0.0005, Loss: 0.0019,  Acc: 100.0000%
Step: 7500, Learn_rate: 0.0005, Loss: 0.0099,  Acc: 100.0000%
Training finished.
T

# 结论
## 虽然 利用正则化 dropout 减小学习率 能够避免过拟合提高训练准确率，但是由于模型自身的问题，测试集的准确率还是比较低，下周我们将用CNN 和RNN模型结构，来提高识别率。